In [1]:
# Real life data

import logging
import threading
import itertools
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
import seaborn as seabornInstance
from sqlalchemy import Column, Integer, String, Float, DateTime, Boolean, func
from iotfunctions import base
from iotfunctions import bif
from iotfunctions import entity
from iotfunctions import metadata
from iotfunctions.metadata import EntityType
from iotfunctions.db import Database
from iotfunctions.enginelog import EngineLogging
from iotfunctions import estimator
from iotfunctions.ui import (UISingle, UIMultiItem, UIFunctionOutSingle,
                 UISingleItem, UIFunctionOutMulti, UIMulti, UIExpression,
                 UIText, UIStatusFlag, UIParameters)
from iotfunctions.dbtables import FileModelStore
from mmfunctions.anomaly import (SaliencybasedGeneralizedAnomalyScore, SpectralAnomalyScore,
                 FFTbasedGeneralizedAnomalyScore, KMeansAnomalyScore, GBMRegressor)
import datetime as dt
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.covariance import MinCovDet
from sklearn import metrics
import scipy as sp
import scipy.fftpack
import skimage as ski  
from skimage import util as skiutil # for nifty windowing
import pyod as pyod
from pyod.utils.data import generate_data
from pyod.utils.data import evaluate_print
from pyod.utils.example import visualize
from pyod.models.knn import KNN
from pyod.models.iforest import IForest
%matplotlib inline
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

EngineLogging.configure_console_logging(logging.INFO)

/home/markus/.local/lib/python3.7/site-packages/iotfunctions/bif.py:1942: UserWarning: IoTCalcSettings is deprecated. Use entity type constants instead of a metadata provider to set entity type properties
  warnings.warn(('IoTCalcSettings is deprecated. Use entity type constants'


In [2]:
credentials={"_id":"2d6dbddcd0ef52f54ab7e7a81f4face6","tenantId":"Monitor-Demo","db2":{"username":"bluadmin","password":"MjZkZTEwN2FjMWY1","databaseName":"BLUDB","port":50000,"httpsUrl":"https://dashdb-enterprise-yp-dal12-125.services.dal.bluemix.net:8443","host":"dashdb-enterprise-yp-dal12-125.services.dal.bluemix.net"},"iotp":{"url":"https://vrvzh6.internetofthings.ibmcloud.com/api/v0002","orgId":"vrvzh6","host":"vrvzh6.messaging.internetofthings.ibmcloud.com","port":8883,"asHost":"api-beta.connectedproducts.internetofthings.ibmcloud.com","apiKey":"a-vrvzh6-am4fwylysp","apiToken":"F+PF@HQhe_N*ecS4gb"},"messageHub":{"brokers":["kafka03-prod02.messagehub.services.us-south.bluemix.net:9093","kafka01-prod02.messagehub.services.us-south.bluemix.net:9093","kafka05-prod02.messagehub.services.us-south.bluemix.net:9093","kafka02-prod02.messagehub.services.us-south.bluemix.net:9093","kafka04-prod02.messagehub.services.us-south.bluemix.net:9093"],"username":"S0Zzzp1zZsF4hotd","password":"S1wyAP2jy9R2cmqcZwSfoqL5gByWSqn9"},"objectStorage":{"region":"global","username":"5c9b5139a83d4f68bd2fe458a2117fac","password":"5b282a68b84d070bd8674da4771998c6cb0743a4a0288129"},"config":{"objectStorageEndpoint":"https://undefined","bos_logs_bucket":"analytics-logs-monitor-demo-9c6aaaf268a1","bos_runtime_bucket":"analytics-runtime-monitor-demo-395729cb6a06","mh_topic_analytics_alerts":"analytics-alerts-Monitor-Demo"}}
db_schema=None
fm = FileModelStore()
db = Database(credentials=credentials, model_store=fm)
print (db)

2020-03-05T11:51:03.612 INFO iotfunctions.dbtables.__init__ Init FileModelStore
2020-03-05T11:51:03.613 INFO iotfunctions.db.__init__ Connection string for SqlAlchemy => db2): db2+ibm_db://bluadmin:MjZkZTEwN2FjMWY1@dashdb-enterprise-yp-dal12-125.services.dal.bluemix.net:50000/BLUDB;


In [3]:
# need a helper function to convert array columns to something easier
from scipy import linalg
def l2norm(df, tcol, col1, col2 = None, col3 = None):
    l2vib = []
    for index, row in df.iterrows():
        l2vib_element = linalg.norm(np.fromstring(row[col1].replace('[',' ').replace(']',''), sep = ','))**2
        if col2 is not None:
            l2vib_element = l2vib_element + \
                            linalg.norm(np.fromstring(row[col2].replace('[',' ').replace(']',''), sep = ','))**2
        if col3 is not None:
            l2vib_element = l2vib_element + \
                            linalg.norm(np.fromstring(row[col3].replace('[',' ').replace(']',''), sep = ','))**2
        l2vib.append(l2vib_element**(1/2))
    df[tcol] = np.asarray(l2vib)
    

def unrollAccel(df):
    l0,l1,l2,l3,l4=[],[],[],[],[]
    for i in df['ACCEL_POWER'].values:
        l0.append(eval(eval(i)[0]))
        l1.append(eval(eval(i)[1]))
        l2.append(eval(eval(i)[2]))
        l3.append(eval(eval(i)[3]))
        l4.append(eval(eval(i)[4]))
    df['accel_power_0'] = np.asarray(l0)
    df['accel_power_1'] = np.asarray(l1)
    df['accel_power_2'] = np.asarray(l2)
    df['accel_power_3'] = np.asarray(l3)
    df['accel_power_4'] = np.asarray(l4)
    
listAttr = ['timestamp','entity','vibrations','rms','accel_speed','accel_power_0','accel_power_1',
            'accel_power_2','accel_power_3','accel_power_4']

In [4]:
# Now we proceed to customer data - GOOD CASE

# Get stuff in
df_input_raw = pd.read_csv('./Armstark04714B6046D5.csv', index_col=False, parse_dates=['RCV_TIMESTAMP_UTC'])
df_input_raw['entity']=df_input_raw['DEVICE_ID']
df_input_raw['timestamp']=df_input_raw['RCV_TIMESTAMP_UTC']

# and sort it by timestamp
df_input_raw = df_input_raw.sort_values(by='timestamp')
df_input_raw = df_input_raw.set_index(['entity','timestamp']).dropna()

l2norm(df_input_raw, 'vibrations', 'VIBRATIONS_XAXIS', 'VIBRATIONS_YAXIS', 'VIBRATIONS_ZAXIS')
l2norm(df_input_raw, 'rms', 'RMS_X', 'RMS_Y', 'RMS_Z')
l2norm(df_input_raw, 'accel_speed', 'ACCEL_SPEED')
unrollAccel(df_input_raw)
#l2norm(df_input_raw, 'accel_power', 'ACCEL_POWER')

df_input = df_input_raw.filter(listAttr, axis=1)
df_input_raw.describe()

/usr/lib/python3/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  
/usr/lib/python3/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  
/usr/lib/python3/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  # Remove the CWD from sys.path while we load stuff.


,vibrations,rms,accel_speed,accel_power_0,accel_power_1,accel_power_2,accel_power_3,accel_power_4
count,8620.000000,8.620000e+03,8620.0,8620.000000,8620.000000,8620.000000,8620.000000,8620.000000
mean,3.759047,1.732051e+00,1.0,2.453523,2.453312,2.452808,2.452053,2.452439
std,0.033272,4.441150e-16,0.0,0.856247,0.857594,0.857678,0.857200,0.856340
min,3.616515,1.732051e+00,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.737696,1.732051e+00,1.0,2.251800,2.251800,2.252000,2.252000,2.251800
50%,3.757949,1.732051e+00,1.0,2.313800,2.313400,2.311800,2.312200,2.312600
75%,3.780154,1.732051e+00,1.0,2.691450,2.692100,2.685100,2.681850,2.682300
max,3.898555,1.732051e+00,1.0,5.325799,5.293200,5.334001,5.278600,5.282000


#### Pandas Profiling

Try Pandas Profiling to get an overview about the data, mostly its distributions and correlations
<br>


In [5]:
# from pandas_profiling import ProfileReport
# profile = ProfileReport(df_input, title='Pandas Profiling Report', html={'style':{'full_width':True}})
# profile.to_widgets()

In [6]:
df_input2 = df_input.copy()
#df_input2.rename(index={'Republic of Korea':'South Korea'},inplace=True)
Indexlist = df_input2.index.to_list()
Indexlist_new = []
for p in Indexlist:
    p_new = ('testentity', p[1])
    Indexlist_new.append(p_new)
df_input2.index = Indexlist_new
df_input = pd.concat([df_input,df_input2])
df_input

vibrations       rms  accel_speed  \
entity       timestamp                                                    
04714B6046D5 2020-01-01 00:02:25.928    3.787826  1.732051          1.0   
             2020-01-01 00:07:26.074    3.776503  1.732051          1.0   
             2020-01-01 00:12:25.663    3.715573  1.732051          1.0   
             2020-01-01 00:17:26.020    3.765538  1.732051          1.0   
             2020-01-01 00:22:25.727    3.725934  1.732051          1.0   
...                                          ...       ...          ...   
testentity   2020-01-31 11:39:23.476    3.725156  1.732051          1.0   
             2020-01-31 11:44:23.497    3.766786  1.732051          1.0   
             2020-01-31 11:49:23.443    3.737750  1.732051          1.0   
             2020-01-31 11:54:23.499    3.771797  1.732051          1.0   
             2020-01-31 11:59:23.501    3.758436  1.732051          1.0   

                                      accel_power_0  accel_power_1  \
entity       timestamp                                               
04714B6046D5 2020-01-01 00:02:25.928         2.2904         2.2824   
             2020-01-01 00:07:26.074         2.2822         2.2978   
             2020-01-01 00:12:25.663         2.2718         2.2908   
             2020-01-01 00:17:26.020         2.2748         2.2932   
             2020-01-01 00:22:25.727         2.3320         2.2980   
...                                             ...            ...   
testentity   2020-01-31 11:39:23.476         2.2778         2.2798   
             2020-01-31 11:44:23.497         2.2734         2.2704   
             2020-01-31 11:49:23.443         2.2666         2.2714   
             2020-01-31 11:54:23.499         2.2722         2.2738   
             2020-01-31 11:59:23.501         2.2760         2.2716   

                                      accel_power_2  accel_power_3  \
entity       timestamp                                               
04714B6046D5 2020-01-01 00:02:25.928         2.2858         2.2956   
             2020-01-01 00:07:26.074         2.2966         2.3504   
             2020-01-01 00:12:25.663         2.2912         2.3044   
             2020-01-01 00:17:26.020         2.2926         2.3146   
             2020-01-01 00:22:25.727         2.2998         2.3212   
...                                             ...            ...   
testentity   2020-01-31 11:39:23.476         2.2676         2.2718   
             2020-01-31 11:44:23.497         2.2694         2.2728   
             2020-01-31 11:49:23.443         2.2712         2.2690   
             2020-01-31 11:54:23.499         2.2694         2.2704   
             2020-01-31 11:59:23.501         2.2678         2.2760   

                                      accel_power_4  
entity       timestamp                               
04714B6046D5 2020-01-01 00:02:25.928         2.3278  
             2020-01-01 00:07:26.074         2.3190  
             2020-01-01 00:12:25.663         2.3232  
             2020-01-01 00:17:26.020         2.3606  
             2020-01-01 00:22:25.727         2.2860  
...                                             ...  
testentity   2020-01-31 11:39:23.476         2.2668  
             2020-01-31 11:44:23.497         2.2670  
             2020-01-31 11:49:23.443         2.2646  
             2020-01-31 11:54:23.499         2.2650  
             2020-01-31 11:59:23.501         2.2744  

[17240 rows x 8 columns]

In [7]:
df_input.describe()

,vibrations,rms,accel_speed,accel_power_0,accel_power_1,accel_power_2,accel_power_3,accel_power_4
count,17240.000000,1.724000e+04,17240.0,17240.000000,17240.000000,17240.000000,17240.000000,17240.000000
mean,3.759047,1.732051e+00,1.0,2.453523,2.453312,2.452808,2.452053,2.452439
std,0.033271,4.441021e-16,0.0,0.856222,0.857569,0.857653,0.857175,0.856315
min,3.616515,1.732051e+00,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.737696,1.732051e+00,1.0,2.251800,2.251800,2.252000,2.252000,2.251800
50%,3.757949,1.732051e+00,1.0,2.313800,2.313400,2.311800,2.312200,2.312600
75%,3.780154,1.732051e+00,1.0,2.691450,2.692100,2.685100,2.681850,2.682300
max,3.898555,1.732051e+00,1.0,5.325799,5.293200,5.334001,5.278600,5.282000


#### Customer suggested a correlation between vibration and acceleration

so let's try to predict (although correlation tests do not really indicate it)

In [11]:
# Run Monitoring's anomaly detector functions
EngineLogging.configure_console_logging(logging.DEBUG)

gbmii = GBMRegressor(features=['accel_power_0','accel_power_1'], max_depth=20, num_leaves=40,
                     n_estimators=4000, learning_rate=0.00001, targets=['vibrations'],
                     predictions=['vibration_pred'])
gbmii.correlation_threshold = 0.001

jobsettings = { 'db': db, 
               '_db_schema': 'public', 'save_trace_to_file' : True}
et = gbmii._build_entity_type(columns = [Column('accel_power_0',Float()), Column('accel_power_1',Float()),
                                        Column('vibrations',Float())], **jobsettings)
gbmii._entity_type = et

# allow training and delete existing models
gbmii.auto_train = True
gbmii.delete_existing_models = True
df_input = gbmii.execute(df=df_input)

2020-03-05T11:54:31.364 DEBUG iotfunctions.enginelog.configure_console_logging Console logging has been configured. Level = 10
2020-03-05T11:54:31.367 DEBUG iotfunctions.metadata.__init__ Initializing new entity type using iotfunctions 2.0.3
2020-03-05T11:54:31.368 DEBUG iotfunctions.util.__init__ Starting trace
2020-03-05T11:54:31.368 DEBUG iotfunctions.util.__init__ Trace name: auto_trace_test_entity_for_GBMRegressor_20200305105431
2020-03-05T11:54:31.369 DEBUG iotfunctions.util.__init__ auto_save None
2020-03-05T11:54:31.370 DEBUG iotfunctions.util.categorize_args categorizing arguments
2020-03-05T11:54:31.371 WARNING iotfunctions.metadata.__init__ Created a logical entity type. It is not connected to a real database table, so it cannot perform any database operations.
2020-03-05T11:54:31.372 DEBUG iotfunctions.metadata.__init__ Initialized entity type 
LocalEntityType:TEST_ENTITY_FOR_GBMREGRESSOR
Functions:
Granularities:
No schedules metadata
2020-03-05T11:54:31.374 DEBUG mmfuncti

/home/markus/.local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/home/markus/.local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:271: UserWarning: The total space of parameters 1 is smaller than n_iter=3. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


2020-03-05T11:54:57.634 DEBUG iotfunctions.base.fit_with_search_cv Used randomize search cross validation to find best hyper parameters for estimator RandomizedSearchCV
2020-03-05T11:54:57.635 INFO iotfunctions.base.find_best_model Trained model no: 0
2020-03-05T11:54:58.051 INFO iotfunctions.base.find_best_model Trained model no: 0 score:0.0026300769101442745
2020-03-05T11:54:58.149 INFO iotfunctions.metadata.test evaluated model model.TEST_ENTITY_FOR_GBMREGRESSOR.GBMRegressor.vibrations.04714B6046D5 with evaluation metric value -2.5316500022798394e-05
2020-03-05T11:54:58.150 DEBUG iotfunctions.base._execute Trained model: {
 "name": "model.TEST_ENTITY_FOR_GBMREGRESSOR.GBMRegressor.vibrations.04714B6046D5",
 "target": "vibrations",
 "features": [
  "accel_power_0",
  "accel_power_1"
 ],
 "estimator_name": "light_gradient_boosted_regressor",
 "eval_metric_name": "r2_score",
 "eval_metric_train": 0.0026300769101442745,
 "eval_metric_test": -2.5316500022798394e-05,
 "trained_date": "2020

/home/markus/.local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/home/markus/.local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:271: UserWarning: The total space of parameters 1 is smaller than n_iter=3. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


2020-03-05T11:55:28.928 DEBUG iotfunctions.base.fit_with_search_cv Used randomize search cross validation to find best hyper parameters for estimator RandomizedSearchCV
2020-03-05T11:55:28.929 INFO iotfunctions.base.find_best_model Trained model no: 0
2020-03-05T11:55:29.290 INFO iotfunctions.base.find_best_model Trained model no: 0 score:0.00266965283223497
2020-03-05T11:55:29.387 INFO iotfunctions.metadata.test evaluated model model.TEST_ENTITY_FOR_GBMREGRESSOR.GBMRegressor.vibrations.testentity with evaluation metric value 0.0001731260893530484
2020-03-05T11:55:29.388 DEBUG iotfunctions.base._execute Trained model: {
 "name": "model.TEST_ENTITY_FOR_GBMREGRESSOR.GBMRegressor.vibrations.testentity",
 "target": "vibrations",
 "features": [
  "accel_power_0",
  "accel_power_1"
 ],
 "estimator_name": "light_gradient_boosted_regressor",
 "eval_metric_name": "r2_score",
 "eval_metric_train": 0.00266965283223497,
 "eval_metric_test": 0.0001731260893530484,
 "trained_date": "2020-03-05T10:55

In [12]:
df_input

vibrations       rms  accel_speed  \
entity       timestamp                                                    
04714B6046D5 2020-01-01 00:02:25.928    3.787826  1.732051          1.0   
             2020-01-01 00:07:26.074    3.776503  1.732051          1.0   
             2020-01-01 00:12:25.663    3.715573  1.732051          1.0   
             2020-01-01 00:17:26.020    3.765538  1.732051          1.0   
             2020-01-01 00:22:25.727    3.725934  1.732051          1.0   
...                                          ...       ...          ...   
testentity   2020-01-31 11:39:23.476    3.725156  1.732051          1.0   
             2020-01-31 11:44:23.497    3.766786  1.732051          1.0   
             2020-01-31 11:49:23.443    3.737750  1.732051          1.0   
             2020-01-31 11:54:23.499    3.771797  1.732051          1.0   
             2020-01-31 11:59:23.501    3.758436  1.732051          1.0   

                                      accel_power_0  accel_power_1  \
entity       timestamp                                               
04714B6046D5 2020-01-01 00:02:25.928         2.2904         2.2824   
             2020-01-01 00:07:26.074         2.2822         2.2978   
             2020-01-01 00:12:25.663         2.2718         2.2908   
             2020-01-01 00:17:26.020         2.2748         2.2932   
             2020-01-01 00:22:25.727         2.3320         2.2980   
...                                             ...            ...   
testentity   2020-01-31 11:39:23.476         2.2778         2.2798   
             2020-01-31 11:44:23.497         2.2734         2.2704   
             2020-01-31 11:49:23.443         2.2666         2.2714   
             2020-01-31 11:54:23.499         2.2722         2.2738   
             2020-01-31 11:59:23.501         2.2760         2.2716   

                                      accel_power_2  accel_power_3  \
entity       timestamp                                               
04714B6046D5 2020-01-01 00:02:25.928         2.2858         2.2956   
             2020-01-01 00:07:26.074         2.2966         2.3504   
             2020-01-01 00:12:25.663         2.2912         2.3044   
             2020-01-01 00:17:26.020         2.2926         2.3146   
             2020-01-01 00:22:25.727         2.2998         2.3212   
...                                             ...            ...   
testentity   2020-01-31 11:39:23.476         2.2676         2.2718   
             2020-01-31 11:44:23.497         2.2694         2.2728   
             2020-01-31 11:49:23.443         2.2712         2.2690   
             2020-01-31 11:54:23.499         2.2694         2.2704   
             2020-01-31 11:59:23.501         2.2678         2.2760   

                                      accel_power_4 vibration_pred  
entity       timestamp                                              
04714B6046D5 2020-01-01 00:02:25.928         2.3278        3.75946  
             2020-01-01 00:07:26.074         2.3190        3.75893  
             2020-01-01 00:12:25.663         2.3232        3.75879  
             2020-01-01 00:17:26.020         2.3606        3.75879  
             2020-01-01 00:22:25.727         2.2860        3.75893  
...                                             ...            ...  
testentity   2020-01-31 11:39:23.476         2.2668        3.75883  
             2020-01-31 11:44:23.497         2.2670        3.75883  
             2020-01-31 11:49:23.443         2.2646        3.75889  
             2020-01-31 11:54:23.499         2.2650        3.75883  
             2020-01-31 11:59:23.501         2.2744        3.75883  

[17240 rows x 9 columns]

In [13]:
print (gbmii._entity_type.db.model_store)

In [2]:
# now predict 
EngineLogging.configure_console_logging(logging.DEBUG)
gbmii = GBMRegressor(features=['accel_power_0'], max_depth=100000, num_leaves=40, threshold=2,
                     n_estimators=1000, learning_rate=0.001, targets=['vibrations'],
                     predictions=['vibration_pred'])

jobsettings = { 'db': db, 
               '_db_schema': 'public', 'save_trace_to_file' : True}
et = gbmii._build_entity_type(columns = [Column('accel_power0',Float())], **jobsettings)
gbmii._entity_type = et

df_input = gbmii.execute(df=df_input)

NameError: name 'EngineLogging' is not defined